**JavaScript Developer?** The [JavaScript version of the `osdatahub` library](https://github.com/OrdnanceSurvey/osdatahub-js) makes it easy to use OS data in the browser or in NodeJS.

# Using the NGD Features API with Python

The National Geographic Database (NGD) Features API provides access to Ordnance Survey's next generation NGD data.  
The NGD Features API is a premium or public sector product only. If you are looking to access free data, you should use the [OS Features API](https://osdatahub.os.uk/docs/wfs/overview).

There are many ways to access the National Geographic Database:
- If you'd like to download data using a GUI, [Select+Build](https://www.ordnancesurvey.co.uk/business-government/products/os-select-build), allows you to download multiple types of features at once, delivered as part of the [OS Data Hub](https://osdatahub.os.uk).
- The [NGD Features API](https://www.ordnancesurvey.co.uk/business-government/products/os-ngd-api-features) allows for programatic access to NGD data in `GeoJSON` format.

For Python developers, the [`osdatahub`](https://github.com/OrdnanceSurvey/osdatahub) Python library wraps almost all APIs in the [OS Data Hub](https://osdatahub.os.uk/) and provides an easy, Pythonic way to get data from Ordnance Survey. We have now added support for the NGD Features API, so you can access OS' latest product and next-generation data with only a few lines of Python code.

## 1. Getting Started

### 1.1 Importing your OS Data Hub API Key
In order to access the NGD, you  must first have an OS Data Hub API key, a notebook explaining the process of registering an account and getting an API key is available to read [here](https://github.com/OrdnanceSurvey/osdatahub/blob/master/Examples/Setting%20up%20an%20API%20key.ipynb). When you're choosing which APIs you want to add to your project, you must select "OS NGD API - Features" to use NGD data.


Once you have an API key, you can load your API key in by either passing it in as a string, or setting it as an environment variable. We recommend using the [`python-dotenv`](https://pypi.org/project/python-dotenv/) library to securely store your API keys in a `.env` file without risk of accidentally committing them in your repo (by adding it to your `.gitignore` file).

To use the `python-dotenv` package, simply:
1. Install with pip: `pip install python-dotenv`
2. Create a `.env` file in the root directory of your project
3. Add your API key to the file: `OS_API_KEY=[YOUR API KEY HERE]`

In [1]:
# Option 1: Assign as a string
# This is less secure, you may commit your API key by accident!
key = "YOUR API KEY HERE"

# Option 2: Use python-dotenv
# This will load environment variables into memory from a .env file
from dotenv import load_dotenv
from os import environ
load_dotenv()
key = environ.get("OS_API_KEY")

Nice one! We can now read the contents of the `key` variable to make sure that worked as expected:

In [ ]:
# 🔎 run this cell to view your API key:
print(f'🔑 Your OS Data Hub key is {key}')

### 1.2 Importing the NGD Module
To import the NGD module from `osdatahub`, write the following:

In [2]:
from osdatahub import NGD

## 2. Discovering Collections
The NGD contains multiple themes and collections:
- A `theme` is a family of `collections`, for example, there is a theme called 'Buildings'.
- A `collection` is a group of features. The Buildings theme contains two collections: 'BuildingLine' and 'BuildingPart'.

You can read more about the NGD Features API's themes and collections in the [Technical Specification](https://osdatahub.os.uk/docs/ofa/technicalSpecification).

To quickly retrieve a list of available data, you can query the API directly using the `NGD` module we previously imported:

In [3]:
ngd_collections = NGD.get_collections()

This will return a JSON object, which we can then interate through to return metadata about each collection.

In [4]:
# 🔎 run this cell to view a list of all available NGD collections
for collection in ngd_collections['collections']:
    print(f' - {collection["title"]} ({collection["id"]})')

 - Building Line (bld-fts-buildingline)
 - Building Part (bld-fts-buildingpart)
 - Named Area (gnm-fts-namedarea)
 - Named Point (gnm-fts-namedpoint)
 - Land (lnd-fts-land)
 - Landform (lnd-fts-landform)
 - Landform Line (lnd-fts-landformline)
 - Landform Point (lnd-fts-landformpoint)
 - Land Point (lnd-fts-landpoint)
 - Site (lus-fts-site)
 - Site Access Location (lus-fts-siteaccesslocation)
 - Site Routing Point (lus-fts-siteroutingpoint)
 - Compound Structure (str-fts-compoundstructure)
 - Structure (str-fts-structure)
 - Structure Line (str-fts-structureline)
 - Structure Point (str-fts-structurepoint)
 - Cartographic Rail Detail (trn-fts-cartographicraildetail)
 - Rail (trn-fts-rail)
 - Road Line (trn-fts-roadline)
 - Road Track Or Path (trn-fts-roadtrackorpath)
 - Connecting Link (trn-ntwk-connectinglink)
 - Connecting Node (trn-ntwk-connectingnode)
 - Ferry Link (trn-ntwk-ferrylink)
 - Ferry Node (trn-ntwk-ferrynode)
 - Ferry Terminal (trn-ntwk-ferryterminal)
 - Path (trn-ntwk-p

## 3. Loading NGD Data into a GeoDataFrame
We'll now walk through the process of acquiring data from the NGD Building Part collection (`bld-fts-buildingpart`) and importing this data into a GeoDataFrame.  

### 3.1 Requesting NGD Data
To get the first 100 features in the collection, you simply need to make an `NGD` object and then run the `query()` method:

In [5]:
collection = "bld-fts-buildingpart"
ngd_building_part = NGD(key, collection)

# make the query and contact the API
features = ngd_building_part.query()

The API returns an OGC-compliant GeoJSON, which is easy to import into other libraries for analysis.


### 3.2 Importing into a GeoDataFrame
If you haven't already, import the `geopandas` library:

In [6]:
import geopandas as gpd

And now, import the NGD data using the `.from_features` function.

In [7]:
gdf = gpd.GeoDataFrame.from_features(features)

Let's take a look at a preview of the data (the `.head` function). As you'll see, NGD data is rich in attribution:

In [8]:
# 🔎 run this cell to preview the contents of the GeoDataFrame
gdf.head()

,geometry,osid,toid,theme,changetype,isobscured,description,versiondate,geometry_area,height_source,...,absoluteheightroofbase,description_updatedate,oslandcover_updatedate,oslanduse_evidencedate,relativeheightroofbase,versionavailabletodate,firstdigitalcapturedate,description_evidencedate,oslandcover_evidencedate,versionavailablefromdate
0,"POLYGON ((-3.97031 55.74255, -3.97019 55.74249...",000000b1-0556-4231-a52a-9b5b8b82dfbf,osgb1000041024621,Buildings,Modified Attributes,False,Building,2022-09-18,59.200650,Ordnance Survey,...,110.85,2006-08-30,2006-08-30,2006-08-30,5.85,None,1991-09-18,2006-08-30,2006-08-30,2022-09-19T00:00:00Z
1,"POLYGON ((1.72756 52.66142, 1.72755 52.66144, ...",00000183-3ae0-4f05-adb8-a5792d09f55f,osgb5000005167584940,Buildings,New,False,Building,2022-08-26,9.499142,Ordnance Survey,...,6.21,2015-10-29,2015-10-29,2015-03-11,0.82,None,2015-11-12,2015-03-11,2015-03-11,2022-08-27T00:00:00Z
2,"POLYGON ((0.19415 51.59241, 0.19411 51.59241, ...",000001d6-8217-4f7a-a70e-3757eb76c9e4,osgb1000000360448,Buildings,New,False,Building,2022-08-26,15.258750,Ordnance Survey,...,37.13,1993-04-01,1993-04-01,1993-04-01,2.43,None,1993-04-01,1993-04-01,1993-04-01,2022-08-27T00:00:00Z
3,"POLYGON ((-1.45472 52.39610, -1.45479 52.39613...",00000208-a659-49f9-a587-ab26cb2b2248,osgb1000017249389,Buildings,New,False,Building,2022-08-26,12.806250,Ordnance Survey,...,71.95,1993-05-01,1993-05-01,1993-05-01,1.96,None,1993-05-01,1993-05-01,1993-05-01,2022-08-27T00:00:00Z
4,"POLYGON ((0.93898 52.81772, 0.93900 52.81767, ...",00000246-42c8-4e23-982e-15613e0fd2fe,osgb1000008341371,Buildings,New,False,Building,2022-08-26,58.280086,Ordnance Survey,...,54.46,2005-04-30,2005-04-30,2005-04-30,3.61,None,1992-03-13,2005-04-30,2005-04-30,2022-08-27T00:00:00Z


You can learn more about importing data from the `osdatahub` package into other common Python libraries [in this example](https://github.com/OrdnanceSurvey/osdatahub/blob/master/Examples/Plotting%20API%20Results%20-%20GeoPandas%2C%20Matplotlib%20and%20Contextily.ipynb).

## 4. Adding Filters

Filters can help you limit the scope of your query, using spatial, temporal and contextual parameters.  
The NGD Features API uses Common Query Language (CQL) to allow you to filter data using the attribution set of each collection.

- Spatial: `extent`  
You can specify any polygon to query by using the `Extent` class. You can learn more about using `Extent` [here](https://github.com/OrdnanceSurvey/osdatahub/blob/master/Examples/Defining%20Extents%20for%20API%20Queries.ipynb)

- Temporal: `start_datetime` and `end_datetime`  
If you want to only get features that have a temporal property, you can specify date ranges to query within. If you want to get features for a single time, simply provide the same argument for both parameters.

- Contextual: `cql_filter`  
The NGD API supports a generic filter grammar called the Common Query Language (CQL) to further filter your query using human readable commands. You can find out more about the operations that the API supports in the Queryables section of the [Technical Specification](https://osdatahub.os.uk/docs/ofa/technicalSpecification). The CQL filter allows you to specify specific properties for features as well as spatial filters.

In addition, the `osdatahub` library permits you to specify the maximum number of features to return (by default, 100 features):

- `max_results`  
Allows you to specify the maximum number of features you'd like to receive. Default is 100.

- `offset`  
Skips past the specified number of features in the collection. Default is 0.

### 4.1 Specifying an Extent (Bounding Box)

In our case, we want to get features that are only within a certain bounding box in Manchester.  
We can specify an extent using the `Extent` submodule:

In [9]:
from osdatahub import Extent

Now we can use the `Extent` module to specify the geometry, and then pass this into the `ngdBuildingPart` query:

In [10]:
extent = Extent.from_bbox((-2.244973,53.476620,-2.237799,53.480525), crs="CRS84")
features = ngd_building_part.query(extent=extent)

# 🔎 run this cell to see the contents of 'features'
features

{'type': 'FeatureCollection',
 'links': [{'href': 'https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingpart/items?limit=100&filter=INTERSECTS%28geometry%2C%20POLYGON%20%28%28-2.237799%2053.47662%2C%20-2.237799%2053.480525%2C%20-2.244973%2053.480525%2C%20-2.244973%2053.47662%2C%20-2.237799%2053.47662%29%29%29&filter-crs=http%3A%2F%2Fwww.opengis.net%2Fdef%2Fcrs%2FOGC%2F1.3%2FCRS84',
   'rel': 'self',
   'type': 'application/geo+json',
   'title': "All features from the 'Building Part' collection"},
  {'href': 'https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingpart/items?offset=100&limit=100&filter=INTERSECTS%28geometry%2C%20POLYGON%20%28%28-2.237799%2053.47662%2C%20-2.237799%2053.480525%2C%20-2.244973%2053.480525%2C%20-2.244973%2053.47662%2C%20-2.237799%2053.47662%29%29%29&filter-crs=http%3A%2F%2Fwww.opengis.net%2Fdef%2Fcrs%2FOGC%2F1.3%2FCRS84',
   'rel': 'next',
   'type': 'application/geo+json',
   'title': 'Next page'}],
 'timeStamp': '2022-11-08T12:44:23

### 4.2 Custom Paging Parameters
By default, the `NGD` module will return a maximum 100 features - starting at zero. This behaviour can be altered by specifying `max_results` and `offset` parameters.  
*We'll build on the previous example, specifying the same extent as before...*

In [11]:
# returns a maximum of 50 results (features 0 to 50)
features = ngd_building_part.query(extent=extent, max_results=50)

# returns a maximum of 50 results (features 100 to 150
features = ngd_building_part.query(extent=extent, offset=100, max_results=50)

# 🔎 run this cell to see the contents of 'features'
features

{'type': 'FeatureCollection',
 'links': [{'href': 'https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingpart/items?offset=100&limit=50&filter=INTERSECTS%28geometry%2C%20POLYGON%20%28%28-2.237799%2053.47662%2C%20-2.237799%2053.480525%2C%20-2.244973%2053.480525%2C%20-2.244973%2053.47662%2C%20-2.237799%2053.47662%29%29%29&filter-crs=http%3A%2F%2Fwww.opengis.net%2Fdef%2Fcrs%2FOGC%2F1.3%2FCRS84',
   'rel': 'self',
   'type': 'application/geo+json',
   'title': "All features from the 'Building Part' collection"},
  {'href': 'https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingpart/items?offset=50&limit=50&filter=INTERSECTS%28geometry%2C%20POLYGON%20%28%28-2.237799%2053.47662%2C%20-2.237799%2053.480525%2C%20-2.244973%2053.480525%2C%20-2.244973%2053.47662%2C%20-2.237799%2053.47662%29%29%29&filter-crs=http%3A%2F%2Fwww.opengis.net%2Fdef%2Fcrs%2FOGC%2F1.3%2FCRS84',
   'rel': 'prev',
   'type': 'application/geo+json',
   'title': 'Previous page'},
  {'href': 'https://api

### 4.3 Applying CQL Filters
Common Query Language (CQL) filters are a handy way in which you can tailor the results of your query to match specific needs. More information on CQL filters can be found [here](https://labs.os.uk/public/osngd/tutorials/articles/#filtering-and-cql). We can pass a filter into the API using the `filter` parameter.

In [12]:
# Building off the same extent covering Manchester, filtering by buildings larger than or equal to 20m^2 and with a maximum height greater than 60m.
# At the time of writing, this returns 7 features (buildings)
features = ngd_building_part.query(extent=extent, cql_filter='geometry_area>=200 AND relativeheightmaximum>60')

# 🔎 run this cell to see the contents of 'features'
features

{'type': 'FeatureCollection',
 'links': [{'href': 'https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingpart/items?limit=100&filter=geometry_area%3E%3D200%20AND%20relativeheightmaximum%3E60%20AND%20INTERSECTS%28geometry%2C%20POLYGON%20%28%28-2.237799%2053.47662%2C%20-2.237799%2053.480525%2C%20-2.244973%2053.480525%2C%20-2.244973%2053.47662%2C%20-2.237799%2053.47662%29%29%29&filter-crs=http%3A%2F%2Fwww.opengis.net%2Fdef%2Fcrs%2FOGC%2F1.3%2FCRS84',
   'rel': 'self',
   'type': 'application/geo+json',
   'title': "All features from the 'Building Part' collection"}],
 'timeStamp': '2022-11-08T12:44:30.285550Z',
 'numberReturned': 7,
 'features': [{'id': '40fa2462-4c01-48d4-bcf1-5179efb56027',
   'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-2.2428973, 53.4772335],
      [-2.2428422, 53.477272],
      [-2.2424657, 53.4775348],
      [-2.2428301, 53.4777217],
      [-2.2428936, 53.4777554],
      [-2.2434293, 53.4773856],
      [-2.2434177, 53.4773

## 5. Conclusion

We hope this short introduction to the `osdatahub` package's new NGD capabilities has been useful!  
You can find further resources to assist you on your OS API development journey at: [https://github.com/OrdnanceSurvey/os-api-resources](https://github.com/OrdnanceSurvey/os-api-resources)